In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from tqdm import tqdm
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
from collections import Counter
import numpy as np

In [117]:
tokens=pd.read_csv('token_review.csv',encoding='utf-8', index_col='Unnamed: 0')
tokens['token_doc']

0       ['좋다', '재질', '하의', '찾기', '적당하다', '어렵다', '수선', ...
1           ['크다', '입다', '확실하다', '거치다', '광택', '사이', '따다']
2       ['추천', '추천', '최고', '최고', '지인', '자다', '사다', '좋다...
3                         ['고급', '지고', '고급', '지다', '스타일']
4       ['빠르다', '배송', '포장', '사용', '예전', '이만', '신발', '좋...
                              ...                        
4536    ['핑크', '자다', '신고', '일반', '양말', '예쁘다', '가격', '사다']
4537         ['여름', '입다', '예쁘다', '선선', '생각', '두껍다', '덥다']
4538                    ['친구', '선물', '두껍다', '선물', '따뜻하다']
4539                    ['편하다', '강추', '강하다', '강하다', '기본']
4540               ['셔츠', '디테일', '입다', '좋다', '살짝', '레이어']
Name: token_doc, Length: 4541, dtype: object

In [119]:
import ast
tokens['token_doc'] = tokens['token_doc'].apply(ast.literal_eval)
tokenized_reviews = tokens['token_doc'].tolist()
word2vec_model = Word2Vec(sentences=tokenized_reviews, window=5,workers=4)
words = list(word2vec_model.wv.index_to_key)
word_vectors = {word: word2vec_model.wv[word] for word in words}

In [139]:
# Convert word vectors to numpy array
word_vectors = np.array([word2vec_model.wv[word] for word in words])

# Perform K-means clustering to reduce words to about 100 clusters
num_clusters = 300
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(word_vectors)

# Get cluster centers and labels
cluster_centers = kmeans.cluster_centers_
labels = kmeans.labels_

# Select a representative word for each cluster
representative_words = []
for i in range(num_clusters):
    # Find words in the current cluster
    cluster_indices = np.where(labels == i)[0]
    cluster_words = [words[idx] for idx in cluster_indices]
    
    # Optionally, choose the word closest to the cluster center as the representative
    center = cluster_centers[i]
    closest_word = min(cluster_words, key=lambda word: np.linalg.norm(word2vec_model.wv[word] - center))
    
    representative_words.append(closest_word)

# Print the representative words
print(representative_words)

['멀다', '물놀이', '커서', '무거워', '핑크', '생각', '블루', '비다', '변형도', '로고', '웜톤', '에스', '다니다', '마르다', '버리다', '반품', '풀다', '처음', '브랜드', '라면', '워싱', '뭔가', '따다', '먹다', '느낌', '딱딱하다', '여름철', '늘어나다', '아쉽다', '내기', '블랙', '냄새', '입히다', '품질', '포켓', '최근', '모양', '어깨', '신발', '핏되', '무지', '약하다', '틀다', '그래픽', '표현', '구입', '신랑', '기대하다', '배송', '카라', '기간', '발도', '거품', '맨투맨', '알맞다', '보이', '클래식', '다르다', '오빠', '버뮤다', '높다', '수영복', '바라다', '사용', '소리', '뜨다', '소재', '추가', '두껍다', '차이', '성비', '화면', '붙다', '편리하다', '오트밀', '스타일', '완성도', '용도', '한여름', '핑크색', '빠르다', '붙이다', '힘드다', '유니크', '선물', '촉감', '귀엽다', '만족', '감다', '항상', '좋아지다', '범용', '모자', '전반', '정핏', '좋아하다', '튼튼하다', '회사', '크기', '촉촉하다', '니트', '가볍다', '걸치다', '셔츠', '번창', '맨날', '순하다', '발색', '스럽지', '공간', '렌즈', '파란색', '중간', '바닥', '쉐입', '자동', '대비', '번째', '어두운', '세미', '말다', '일가', '다드', '따뜻하다', '넓다', '가격', '맞다', '무겁다', '폴로', '지지', '고급', '미리', '수납', '색상', '주문', '입어보다', '찾다', '팬츠', '잡히다', '크롭', '포인트', '향수', '베이지', '휘뚜루', '해도', '제품', '후회', '카드', '만족하다', '건지다', '색감', '상태', '기도', '선명하다', '마음', '게요

In [ ]:
reviews=pd.read_csv('review_40000_data.csv', encoding='utf-8')
reviews.head()

In [ ]:
combined_reviews = reviews.groupby('user_id')['review_text'].apply(lambda x: ' '.join(set(x))).reset_index()
combined_reviews.head()

In [113]:
combined_reviews['review_text']=combined_reviews['review_text'].str.replace("[^ㄱ-하-ㅣ가-힣 ]","")
combined_reviews['review_text'].dropna(inplace=True)

C:\Users\duwjd\AppData\Local\Temp\ipykernel_9276\2745201745.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  combined_reviews['review_text']=combined_reviews['review_text'].str.replace("[^ㄱ-하-ㅣ가-힣 ]","")


In [115]:
from hanspell import spell_checker

spell=[]
for i in tqdm(range(len(combined_reviews['review_text']))):
    try:
        sent = combined_reviews.iloc[i,1]
        spelled_sent = spell_checker.check(sent)
        hanspell_sent = spelled_sent.checked
        spell.append(hanspell_sent)
    except:
        spell.append("")

combined_reviews['spell_doc']=spell
combined_reviews

100%|███████████████████████████████████████████████████████████████████████████| 38396/38396 [02:04<00:00, 309.14it/s]


,user_id,review_text,spell_doc
0,!!??!?,배송도 빠르고 저렴하게 구매했어요 키로 개월 남아에게도 잘어울리네요 아기기 잘머거서...,
1,!!ㅇㅈㅁ!!,옷이 부드럽고 두꺼워서 잘 입고 있어요 그리고 편해서 좋네요 너무 잘 입고 있습니다...,
2,!i!i!,발수도 잘 되고 양산으로서의 기능도 확실해요 무엇보다도 접고 펼 때 부드럽게 움직여...,
3,!•c•!,보풀이 짱 심합니다 입고 나간 날은 항상 제거해주고 있어요 귀찮긴 한데 이뻐서 ㄱㅊ...,
4,!ㅇㅁㅇ!,색감이 너무 좋고 재질도 만족합니다 추천합니다 배송 빠르고 만족합니다 틴트라서 투과...,
...,...,...,...
38391,힣헿훟힣,바를 때 조금 따가워요전 피부장벽개선후 발라야 효과가있을고같아요 굉장히 이쁩니다 원...,
38392,劉 周 城,옷이 예쁘고 잘맞아요 잘입겠습니다 느아으아앙앙아 깡 형님 믿고 구입 완전 도톰하고 ...,
38393,柳,평소 신는데 이것도 하니 잘 맞습니다 혜자스러운 금액 감사합니다 한 해는 끄떡 없겠...,
38394,｡｡,공병을 못구해서 아직 못써봤지만 즐겨보는 유튜버가 쓰는 제품이라 기대가 됩니다 가볍...,


In [116]:
combined_reviews['spell_doc'].replace('', pd.NA, inplace=True)
combined_reviews['combined'] = combined_reviews['spell_doc'].fillna(combined_reviews['review_text'])
combined_reviews

,user_id,review_text,spell_doc,combined
0,!!??!?,배송도 빠르고 저렴하게 구매했어요 키로 개월 남아에게도 잘어울리네요 아기기 잘머거서...,NaN,배송도 빠르고 저렴하게 구매했어요 키로 개월 남아에게도 잘어울리네요 아기기 잘머거서...
1,!!ㅇㅈㅁ!!,옷이 부드럽고 두꺼워서 잘 입고 있어요 그리고 편해서 좋네요 너무 잘 입고 있습니다...,NaN,옷이 부드럽고 두꺼워서 잘 입고 있어요 그리고 편해서 좋네요 너무 잘 입고 있습니다...
2,!i!i!,발수도 잘 되고 양산으로서의 기능도 확실해요 무엇보다도 접고 펼 때 부드럽게 움직여...,NaN,발수도 잘 되고 양산으로서의 기능도 확실해요 무엇보다도 접고 펼 때 부드럽게 움직여...
3,!•c•!,보풀이 짱 심합니다 입고 나간 날은 항상 제거해주고 있어요 귀찮긴 한데 이뻐서 ㄱㅊ...,NaN,보풀이 짱 심합니다 입고 나간 날은 항상 제거해주고 있어요 귀찮긴 한데 이뻐서 ㄱㅊ...
4,!ㅇㅁㅇ!,색감이 너무 좋고 재질도 만족합니다 추천합니다 배송 빠르고 만족합니다 틴트라서 투과...,NaN,색감이 너무 좋고 재질도 만족합니다 추천합니다 배송 빠르고 만족합니다 틴트라서 투과...
...,...,...,...,...
38391,힣헿훟힣,바를 때 조금 따가워요전 피부장벽개선후 발라야 효과가있을고같아요 굉장히 이쁩니다 원...,NaN,바를 때 조금 따가워요전 피부장벽개선후 발라야 효과가있을고같아요 굉장히 이쁩니다 원...
38392,劉 周 城,옷이 예쁘고 잘맞아요 잘입겠습니다 느아으아앙앙아 깡 형님 믿고 구입 완전 도톰하고 ...,NaN,옷이 예쁘고 잘맞아요 잘입겠습니다 느아으아앙앙아 깡 형님 믿고 구입 완전 도톰하고 ...
38393,柳,평소 신는데 이것도 하니 잘 맞습니다 혜자스러운 금액 감사합니다 한 해는 끄떡 없겠...,NaN,평소 신는데 이것도 하니 잘 맞습니다 혜자스러운 금액 감사합니다 한 해는 끄떡 없겠...
38394,｡｡,공병을 못구해서 아직 못써봤지만 즐겨보는 유튜버가 쓰는 제품이라 기대가 됩니다 가볍...,NaN,공병을 못구해서 아직 못써봤지만 즐겨보는 유튜버가 쓰는 제품이라 기대가 됩니다 가볍...


In [117]:
combined_reviews.drop('spell_doc', axis=1,inplace=True)

In [121]:

# 불용어 정의
with open('stopwords.txt', 'r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")

okt = Okt()

tokenized_data = []
for sentence in tqdm(combined_reviews['combined']):
    tokenized_sentence = okt.morphs(sentence, stem=True)  # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords and len(word) >= 2]  # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

token_counter = Counter(token for sentence in tokenized_data for token in sentence)

filtered_tokenized_data = []
for sentence in tokenized_data:
    filtered_sentence = [word for word in sentence if token_counter[word] > 3]
    filtered_tokenized_data.append(filtered_sentence)

combined_reviews['token_doc'] = filtered_tokenized_data
combined_reviews

100%|██████████████████████████████████████████████████████████████████████████| 38396/38396 [4:49:37<00:00,  2.21it/s]


,user_id,review_text,combined,token_doc
0,!!??!?,배송도 빠르고 저렴하게 구매했어요 키로 개월 남아에게도 잘어울리네요 아기기 잘머거서...,배송도 빠르고 저렴하게 구매했어요 키로 개월 남아에게도 잘어울리네요 아기기 잘머거서...,"[배송, 빠르다, 저렴하다, 구매, 키로, 개월, 남아, 에게도, 어울리다, 아기,..."
1,!!ㅇㅈㅁ!!,옷이 부드럽고 두꺼워서 잘 입고 있어요 그리고 편해서 좋네요 너무 잘 입고 있습니다...,옷이 부드럽고 두꺼워서 잘 입고 있어요 그리고 편해서 좋네요 너무 잘 입고 있습니다...,"[부드럽다, 두껍다, 자다, 입다, 편하다, 좋다, 자다, 입다, 허리, 크다, 빨..."
2,!i!i!,발수도 잘 되고 양산으로서의 기능도 확실해요 무엇보다도 접고 펼 때 부드럽게 움직여...,발수도 잘 되고 양산으로서의 기능도 확실해요 무엇보다도 접고 펼 때 부드럽게 움직여...,"[수도, 자다, 양산, 으로서의, 기능, 확실하다, 보다도, 접다, 펴다, 부드럽다..."
3,!•c•!,보풀이 짱 심합니다 입고 나간 날은 항상 제거해주고 있어요 귀찮긴 한데 이뻐서 ㄱㅊ...,보풀이 짱 심합니다 입고 나간 날은 항상 제거해주고 있어요 귀찮긴 한데 이뻐서 ㄱㅊ...,"[보풀, 심하다, 입다, 나가다, 항상, 제거, 해주다, 귀찮다, 이쁘다, ㄱㅊㄱㅊ..."
4,!ㅇㅁㅇ!,색감이 너무 좋고 재질도 만족합니다 추천합니다 배송 빠르고 만족합니다 틴트라서 투과...,색감이 너무 좋고 재질도 만족합니다 추천합니다 배송 빠르고 만족합니다 틴트라서 투과...,"[색감, 좋다, 재질, 만족하다, 추천, 배송, 빠르다, 만족하다, 틴트, 좋다, ..."
...,...,...,...,...
38391,힣헿훟힣,바를 때 조금 따가워요전 피부장벽개선후 발라야 효과가있을고같아요 굉장히 이쁩니다 원...,바를 때 조금 따가워요전 피부장벽개선후 발라야 효과가있을고같아요 굉장히 이쁩니다 원...,"[바르다, 따갑다, 요전, 피부, 장벽, 선후, 발라, 효과, 굉장하다, 이쁘다, ..."
38392,劉 周 城,옷이 예쁘고 잘맞아요 잘입겠습니다 느아으아앙앙아 깡 형님 믿고 구입 완전 도톰하고 ...,옷이 예쁘고 잘맞아요 잘입겠습니다 느아으아앙앙아 깡 형님 믿고 구입 완전 도톰하고 ...,"[예쁘다, 잘맞다, 입다, 으아, 형님, 믿다, 구입, 도톰하다, 좋다, 날씨, 구..."
38393,柳,평소 신는데 이것도 하니 잘 맞습니다 혜자스러운 금액 감사합니다 한 해는 끄떡 없겠...,평소 신는데 이것도 하니 잘 맞습니다 혜자스러운 금액 감사합니다 한 해는 끄떡 없겠...,"[평소, 신다, 하니, 자다, 맞다, 혜자, 스러운, 금액, 감사하다, 여름, 입다..."
38394,｡｡,공병을 못구해서 아직 못써봤지만 즐겨보는 유튜버가 쓰는 제품이라 기대가 됩니다 가볍...,공병을 못구해서 아직 못써봤지만 즐겨보는 유튜버가 쓰는 제품이라 기대가 됩니다 가볍...,"[공병, 구해, 즐기다, 유튜버, 쓰다, 제품, 기대, 가볍다, 바르다, 좋다, 두..."


In [69]:
df1=pd.read_csv('review_token_40000.csv',encoding='utf-8',index_col='Unnamed: 0')
df1

,user_id,token_doc
0,!!??!?,"['배송', '빠르다', '저렴하다', '구매', '키로', '개월', '남아', ..."
1,!!ㅇㅈㅁ!!,"['부드럽다', '두껍다', '입다', '편하다', '좋다', '입다', '허리',..."
2,!i!i!,"['수도', '양산', '으로서의', '기능', '확실하다', '보다도', '접다'..."
3,!•c•!,"['보풀', '심하다', '입다', '나가다', '항상', '제거', '해주다', ..."
4,!ㅇㅁㅇ!,"['색감', '좋다', '재질', '만족하다', '추천', '배송', '빠르다', ..."
...,...,...
38391,힣헿훟힣,"['바르다', '따갑다', '요전', '피부', '장벽', '선후', '발라', '..."
38392,劉 周 城,"['예쁘다', '잘맞다', '입다', '으아', '형님', '믿다', '구입', '..."
38393,柳,"['평소', '신다', '하니', '맞다', '혜자', '스러운', '금액', '감..."
38394,｡｡,"['공병', '구해', '즐기다', '유튜버', '제품', '기대', '가볍다', ..."


In [81]:
df1['token_doc'] = df1['token_doc'].apply(lambda x: ast.literal_eval(x))

In [99]:
replacements = {
    '버핏' : '오버핏',
    '빨르다' : '빠르다',
    '디테' : '디테일',
    '이쁘다' : '예쁘다',
    '쁘네' : '예쁘네',
    '휘뚜루' : '휘뚜루',
    '뚜루' : '휘뚜루',
    '휘뚤마뚤' : '휘뚜루',
    '성도' : '완성도',
    '두운' : '어두운',
    '투맨' : '맨투맨',
    '조아' : '좋다',
    '레터' : '레터링',
    '웨이' : '투웨이',
    '데일' : '데일리',
    '쁜데' : '예쁜데',
    '쁩니' : '예쁩니다',
    '핏입니' : '핏입니다',
    '가성' : '가성비',
    '기대하다' : '기대',
    '두툼해' : '두껍다',
    '도톰하다' : '두껍다',
    '만족스럽다' : '만족',
    '만족하다': '만족',
    '맨날' : '매일',
    '무신' : '무신사',
    '무탠' : '무탠다드',
    '보리' : '시보리',
    '비다' : '빠지다',
    '사고' : '사다',
    '사면': '사다',
    '사보다': '사다',
    '사서': '사다',
    '사주다': '사다',
    '사이' : '사이즈',
    '색도' : '색깔',
    '색상' : '색깔',
    '신고':'신다',
    '신기':'신다',
    '요새' : '요즘',
    '입어보다':'입다',
    '적당':'적당하다',
    '전반' : '전체',
    '좋아하다' : '좋다',
    '커서':'크다',
    '편안하다':'편하다',
    '편이':'편하다',
    '프린트':'프린팅',
    '필요하다':'필수',
    '강력':'강추',
    '검정':'색상',
    '봄':'계절',
    '여름':'계절',
    '가을':'계절',
    '겨울':'계절',
    '구입':'구매',
    '그레이':'색상',
    '기장':'길이',
    '라이트':'색감',
    '다크':'색감',
    '어둡다':'색감',
    '라지':'사이즈',
    '미듐':'사이즈',
    '스몰':'사이즈',
    '브라운':'색깔',
    '비니':'모자',
    '빨리':'빠르다',
    '사다':'구매',
    '실버':'색깔',
    '아이보리':'색깔',
    '오버':'오버핏',
    '차콜':'색깔',
    '초여름':'계절',
    '친구':'지인',
    '컬러':'색깔',
    '크게':'크기',
    '크림':'색깔',
    '핏되':'슬림',
    '붙다':'슬림',
    '핑크':'색깔',
    '한여름':'계절',
    '화이트':'색깔',
    '흰색':'색깔',
    '희다':'색깔',
    '끌리다':'길다',
    '넓다':'넉넉하다',
    '여유':'넉넉하다',
    '오버핏':'넉넉하다',
    '네이비':'색깔',
    '그대로':'동일하다',
    '똑같다':'동일하다',
    '딱이다':'딱맞다',
    '맞다':'딱맞다',
    '밝다':'색감',
    '스타일링':'스타일',
    '실물':'실제',
    '은은하다':'은근',
    '입히다':'입다',
    '좁다':'슬림',
    '지퍼':'스트링',
    '특유':'특이하다',
    '팬츠':'바지'
    
}

delete_words = {
    '자다', '쓰다','갈다','감다','개다','거품','그것','기다','깔다','꾸다','나가다','나인','너무나','넣다','늘다','다드','돌다','돌리다','듭니','따다',
    '리하','마뚜루','만에','물다','바르다','뿌리다','스럽게','쏘다','쓸다','쓰기','아무','아래','아주','에도','에어도','에서도','에스','웨어','으로도','이라서',
    '이만','이며','이지','일가','일리','일해','자수','잘살다','잡다','질도','찌다','추다','쿠어','편입','해보다','해봤다','해주다','가방','가죽','공간','남아',
    '남자친구','남편','높다','다니다','다운','달라','대비','대신','동생','되게','드네','드리다','들어가다','들어서다','만들다','많다','말다','말리다','맛있다','매트',
    '먹다','메다','물다','바닥','박다','반스','발림','보기','보내다','보이','사람','사실','살다','생각','서다','성비','세미','선택','수고','쉬다','신랑','신축','엄마',
    '아기','아들','아래','아무','여자친구','와이프','용도','유지','의사','이용','이유','저녁','전투','접다','전체','주문','줍다','지도','질도','찌다','찍다','출근','케이스',
    '쿠션','타이','톡톡하다','파다','패드','하루','학년','햇빛','향수','가능하다','갖다','건조기','고르다','기간','기분','기준','단독','두루','등산','뜨다','러닝',
    '마음','마르다','물건','박스','빨다','사용','사실','수납','수분','아디다스','아주','여행','연출','용량','운동','일반','제품','조절','참고','찾다','추가','촉촉하다','커플','커버','카드',
    '폴로','활동','효과','흡수수'
,
}

def replace_and_delete_tokens(tokens, replacements, delete_words):
    return [replacements.get(token, token) for token in tokens if token not in delete_words]

# Apply function to 'token_doc' column
df1['token_doc'] = df1['token_doc'].apply(lambda x: replace_and_delete_tokens(x, replacements, delete_words))
df1

,user_id,token_doc,keywords_by_frequency,keywords_by_similarity
0,!!??!?,"[배송, 빠르다, 저렴하다, 구매, 키로, 개월, 에게도, 어울리다, 저렴하다, 구...",구매 입히다 저렴하다,느낌 예쁘다 코디
1,!!ㅇㅈㅁ!!,"[부드럽다, 두껍다, 입다, 편하다, 좋다, 입다, 허리, 크다, 빨래, 허리, 삼...",예쁘다 입다 좋다,예쁘다 코디 사다
2,!i!i!,"[수도, 양산, 으로서의, 기능, 확실하다, 보다도, 펴다, 부드럽다, 움직, 좋다...",좋다 입다 예쁘다,느낌 코디 사다
3,!•c•!,"[보풀, 심하다, 입다, 항상, 제거, 귀찮다, 예쁘다, ㄱㅊㄱㅊ, 예쁘다, 기모,...",좋다 예쁘다 입다,느낌 예쁘다 코디
4,!ㅇㅁㅇ!,"[색깔, 좋다, 재질, 만족, 추천, 배송, 빠르다, 만족, 틴트, 좋다, 배송, ...",배송 만족 빠르다,귀엽다 느낌 좋다
...,...,...,...,...
38391,힣헿훟힣,"[따갑다, 요전, 피부, 장벽, 선후, 발라, 굉장하다, 예쁘다, 원단, 얇다, 프...",좋다 입다 크다,느낌 사다 코디
38392,劉 周 城,"[예쁘다, 잘맞다, 입다, 으아, 형님, 믿다, 구매, 두껍다, 좋다, 날씨, 구매...",예쁘다 입다 구매,예쁘다 구매 코디
38393,柳,"[평소, 신다, 하니, 딱맞다, 혜자, 스러운, 금액, 감사하다, 계절, 입다, 적...",구매 만족 좋다,느낌 사다 괜찮다
38394,｡｡,"[공병, 구해, 즐기다, 유튜버, 기대, 가볍다, 좋다, 두껍다, 얇다, 이제야, ...",좋다 구매 배송,느낌 사다 귀엽다


In [101]:
df1['token_doc'] = df1['token_doc'].apply(lambda x: eval(x) if isinstance(x, str) else x)

def filter_tokens(tokens, model):
    return [token for token in tokens if token in model.wv]

filtered_reviews = [filter_tokens(review, word2vec_model) for review in df1['token_doc']]

def get_keywords_by_frequency(tokens, top_n=5):
    frequency = Counter(tokens)
    return [keyword for keyword, _ in frequency.most_common(top_n)]

def get_average_vector(tokens, model):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

def find_closest_words(vector, model, topn=3):
    return [word for word, _ in model.wv.similar_by_vector(vector, topn=topn)]

# Extract keywords for each review
keywords_by_frequency = [get_keywords_by_frequency(review) for review in filtered_reviews]
keywords_by_similarity = []

for review in tqdm(filtered_reviews):
    average_vector = get_average_vector(review, word2vec_model)
    similar_keywords = find_closest_words(average_vector, word2vec_model)
    keywords_by_similarity.append(similar_keywords)

# Convert keywords to string format for DataFrame
keywords_by_frequency_str = [' '.join(keywords) for keywords in keywords_by_frequency]
keywords_by_similarity_str = [' '.join(keywords) for keywords in keywords_by_similarity]

# Add new columns to the DataFrame
df1['keywords_by_frequency'] = keywords_by_frequency_str
df1['keywords_by_similarity'] = keywords_by_similarity_str
df1

100%|███████████████████████████████████████████████████████████████████████████| 38396/38396 [00:39<00:00, 964.23it/s]


,user_id,token_doc,keywords_by_frequency,keywords_by_similarity
0,!!??!?,"[배송, 빠르다, 저렴하다, 구매, 키로, 개월, 에게도, 어울리다, 저렴하다, 구...",구매 입다 저렴하다 좋다 크다,구매 느낌 사다
1,!!ㅇㅈㅁ!!,"[부드럽다, 두껍다, 입다, 편하다, 좋다, 입다, 허리, 크다, 빨래, 허리, 삼...",예쁘다 입다 좋다 크다 편하다,예쁘다 코디 생각
2,!i!i!,"[수도, 양산, 으로서의, 기능, 확실하다, 보다도, 펴다, 부드럽다, 움직, 좋다...",좋다 입다 색깔 예쁘다 편하다,느낌 코디 생각
3,!•c•!,"[보풀, 심하다, 입다, 항상, 제거, 귀찮다, 예쁘다, ㄱㅊㄱㅊ, 예쁘다, 기모,...",좋다 예쁘다 입다 구매 사이즈,느낌 예쁘다 코디
4,!ㅇㅁㅇ!,"[색깔, 좋다, 재질, 만족, 추천, 배송, 빠르다, 만족, 틴트, 좋다, 배송, ...",배송 만족 빠르다 좋다 색깔,귀엽다 느낌 생각
...,...,...,...,...
38391,힣헿훟힣,"[따갑다, 요전, 피부, 장벽, 선후, 발라, 굉장하다, 예쁘다, 원단, 얇다, 프...",좋다 입다 크다 두껍다 예쁘다,느낌 사다 생각
38392,劉 周 城,"[예쁘다, 잘맞다, 입다, 으아, 형님, 믿다, 구매, 두껍다, 좋다, 날씨, 구매...",예쁘다 입다 구매 좋다 믿다,예쁘다 구매 코디
38393,柳,"[평소, 신다, 하니, 딱맞다, 혜자, 스러운, 금액, 감사하다, 계절, 입다, 적...",구매 만족 좋다 입다 사이즈,느낌 사다 코디
38394,｡｡,"[공병, 구해, 즐기다, 유튜버, 기대, 가볍다, 좋다, 두껍다, 얇다, 이제야, ...",좋다 구매 배송 빠르다 예쁘다,느낌 사다 귀엽다


In [103]:
save_review=df1.loc[:,['user_id','keywords_by_frequency','keywords_by_similarity']]
save_review.to_csv('review_keyword_40000.csv', encoding='utf-8')

In [154]:
# token_review=combined_reviews.loc[:,['user_id','token_doc']]
# token_review.to_csv('review_token_40000.csv', encoding='utf-8')